In [54]:
from bs4 import BeautifulSoup, Comment
from datetime import datetime, timedelta, timezone

import requests
import pandas as pd

In [15]:
datetime = datetime.today()
date = datetime.strftime("%Y%m%d")
date

'20240501'

In [16]:
link = f"https://www.espn.com/nba/schedule/_/date/{date}"

In [17]:
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

page = requests.get(link, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

In [18]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<!-- ESPNFITT | 26521826c9f4 | 8445829 | e357aeec | www.espn.com | Wed, 01 May 2024 18:34:17 GMT -->
<script async="" src="https://dcf.espn.com/TWDC-DTCI/prod/Bootstrap.js"></script>
<script data-nr-pct="1" data-nr-type="legacy">(() => {function gc(n){n=document.cookie.match("(^|;) ?"+n+"=([^;]*)(;|$)");return n?n[2]:null}function sc(n){document.cookie=n}function smpl(n){n/=100;return!!n&&Math.random()<=n}var _nr=!1,_nrCookie=gc("_nr");null!==_nrCookie?"1"===_nrCookie&&(_nr=!0):smpl(1)?(_nr=!0,sc("_nr=1; path=/")):(_nr=!1,sc("_nr=0; path=/"));_nr && (() => {;window.NREUM||(NREUM={});NREUM.init={distributed_tracing:{enabled:true},privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-data.net"]}};;NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","sa":1,"licenseKey":"d1734eda45","applicationID":"350640159"};__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.

In [24]:
todays_games = soup.find("div", class_ = "ScheduleTables").find("tbody")
todays_games

<tbody class="Table__TBODY"><tr class="Table__TR Table__TR--sm Table__even" data-idx="0"><td class="events__col Table__TD"><div class="matchTeams"><span class="Table__Team away"><a class="AnchorLink" data-track-linkid="nba:schedule:team" data-track-navmethod="schedule" href="/nba/team/_/name/phi/philadelphia-76ers" tabindex="0"></a><a class="AnchorLink" data-track-linkid="nba:schedule:team" data-track-navmethod="schedule" href="/nba/team/_/name/phi/philadelphia-76ers" tabindex="0">Philadelphia</a></span></div><span class="gameNote pt3">East 1st Round - Game 5, NY leads series 3-1</span></td><td class="colspan__col Table__TD"><div class="local flex items-center"><span class="at">  @  </span><span class="Table__Team"><a class="AnchorLink" data-track-linkid="nba:schedule:team" data-track-navmethod="schedule" href="/nba/team/_/name/ny/new-york-knicks" tabindex="0"></a><a class="AnchorLink" data-track-linkid="nba:schedule:team" data-track-navmethod="schedule" href="/nba/team/_/name/ny/new-y

In [27]:
data = []
rows = todays_games.find_all('tr')
for row in rows:
    cols = row.find_all('span')
    game = []
    # Skip tickets column
    for col in cols[:-1]:
        col = col.text.strip()
        if col != "@":
            game.append(col)
    data.append(game)

In [28]:
data

[['Philadelphia', 'East 1st Round - Game 5, NY leads series 3-1', 'New York'],
 ['Orlando', 'East 1st Round - Game 5, Series tied 2-2', 'Cleveland'],
 ['Indiana', 'East 1st Round - Game 5, IND leads series 3-1', 'Milwaukee']]

In [47]:
today = datetime.today()
tomorrow = today + timedelta(days=1)

In [48]:
today = today.strftime("%Y-%m-%dT00:00:00Z")
tomorrow = tomorrow.strftime("%Y-%m-%dT04:00:00Z")
print(today, tomorrow)

2024-05-09T00:00:00Z 2024-05-10T04:00:00Z


In [19]:
resp = requests.get("https://api.the-odds-api.com/v4/sports/basketball_nba/odds",
                    params={
                        "api_key" : "18d4f5a57823e4304f11070456e9b30e",
                        "regions" : "us",
                        "markets" : "h2h,spreads,totals",
                        "oddsFormat" : "american",
                        "dateFormat" : "iso",
                        "bookmakers" : "fanduel",
                        "commenceTimeFrom" : str(today),
                        "commenceTimeTo" : tomorrow
                    })

In [20]:
if resp.status_code != 200:
    print(f'Failed to get odds: status_code {resp.status_code}, response body {resp.text}')

else:
    odds_json = resp.json()
    #print('Number of events:', len(odds_json))
    # Check the usage quota
    #print('Remaining requests', resp.headers['x-requests-remaining'])
    #print('Used requests', resp.headers['x-requests-used'])

Number of events: 2
[{'id': 'e395974295e16027bfeaf8e1d776ddee', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2024-05-09T23:00:00Z', 'home_team': 'Boston Celtics', 'away_team': 'Cleveland Cavaliers', 'bookmakers': [{'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2024-05-09T16:18:25Z', 'markets': [{'key': 'h2h', 'last_update': '2024-05-09T16:18:25Z', 'outcomes': [{'name': 'Boston Celtics', 'price': -950}, {'name': 'Cleveland Cavaliers', 'price': 640}]}, {'key': 'spreads', 'last_update': '2024-05-09T16:18:25Z', 'outcomes': [{'name': 'Boston Celtics', 'price': -112, 'point': -13.5}, {'name': 'Cleveland Cavaliers', 'price': -108, 'point': 13.5}]}, {'key': 'totals', 'last_update': '2024-05-09T16:18:25Z', 'outcomes': [{'name': 'Over', 'price': -110, 'point': 212.5}, {'name': 'Under', 'price': -110, 'point': 212.5}]}]}]}, {'id': '56493f78ffa4a7745c6c011bcc635091', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2024-05-10T01:30:00Z', 'hom

In [24]:
resp.text

'[{"id":"e395974295e16027bfeaf8e1d776ddee","sport_key":"basketball_nba","sport_title":"NBA","commence_time":"2024-05-09T23:00:00Z","home_team":"Boston Celtics","away_team":"Cleveland Cavaliers","bookmakers":[{"key":"fanduel","title":"FanDuel","last_update":"2024-05-09T16:18:25Z","markets":[{"key":"h2h","last_update":"2024-05-09T16:18:25Z","outcomes":[{"name":"Boston Celtics","price":-950},{"name":"Cleveland Cavaliers","price":640}]},{"key":"spreads","last_update":"2024-05-09T16:18:25Z","outcomes":[{"name":"Boston Celtics","price":-112,"point":-13.5},{"name":"Cleveland Cavaliers","price":-108,"point":13.5}]},{"key":"totals","last_update":"2024-05-09T16:18:25Z","outcomes":[{"name":"Over","price":-110,"point":212.5},{"name":"Under","price":-110,"point":212.5}]}]}]},{"id":"56493f78ffa4a7745c6c011bcc635091","sport_key":"basketball_nba","sport_title":"NBA","commence_time":"2024-05-10T01:30:00Z","home_team":"Oklahoma City Thunder","away_team":"Dallas Mavericks","bookmakers":[{"key":"fanduel",

In [62]:
est_offset = timedelta(hours=-6)
original_timezone = timezone.utc

In [75]:
for json in odds_json:
    home_team = json['home_team']
    away_team = json['away_team']
    time = datetime.fromisoformat(json['commence_time']).astimezone(original_timezone) + est_offset
    bm = json['bookmakers']

    # Will need to change if more sports books are added
    bookmaker = bm[0]['title']
    markets = bm[0]['markets']

    for m in markets:
        key = m['key']
        for team in m['outcomes']:
            if team['name'] == home_team:
                if key == 'h2h':
                    home_h2h_price = team['price']
                elif key == 'spreads':
                    home_spreads_price = team['price']
                    home_spreads_points = team['point']
            elif team['name'] == away_team:
                if key == 'h2h':
                    away_h2h_price = team['price']
                elif key == 'spreads':
                    away_spreads_price = team['price']
                    away_spreads_points = team['point']
            elif team['name'] == "Over":
                over_price = team['price']
                over_point = team['point']
            else:
                under_price = team['price']
                under_point = team['point']


    print(time)
    print(home_team, home_h2h_price, home_spreads_price, home_spreads_points)
    print(away_team, away_h2h_price, away_spreads_price, away_spreads_points)
    print(over_price, over_point)
    print(under_price, under_point)
    print()



<class 'datetime.datetime'>
Boston Celtics -950 -112 -13.5
Cleveland Cavaliers 640 -108 13.5
-110 212.5
-110 212.5

<class 'datetime.datetime'>
Oklahoma City Thunder -200 -110 -5.0
Dallas Mavericks 168 -110 5.0
-108 218.0
-112 218.0



In [22]:
import json

print(json.dumps(odds_json, indent=4))

[
    {
        "id": "e395974295e16027bfeaf8e1d776ddee",
        "sport_key": "basketball_nba",
        "sport_title": "NBA",
        "commence_time": "2024-05-09T23:00:00Z",
        "home_team": "Boston Celtics",
        "away_team": "Cleveland Cavaliers",
        "bookmakers": [
            {
                "key": "fanduel",
                "title": "FanDuel",
                "last_update": "2024-05-09T16:18:25Z",
                "markets": [
                    {
                        "key": "h2h",
                        "last_update": "2024-05-09T16:18:25Z",
                        "outcomes": [
                            {
                                "name": "Boston Celtics",
                                "price": -950
                            },
                            {
                                "name": "Cleveland Cavaliers",
                                "price": 640
                            }
                        ]
                    },
      